# Part 1: Web Scraping: IMDB TV Shows

615 Titles from IMDB Advanced search based on: TV series, 1990 - 2017, USA, English,
rating:1-10, minumum number of votes: 5.000

In [468]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import itertools
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [469]:
url = "http://www.imdb.com/search/title?count=250&countries=us&languages=en&num_votes=10000,50000&release_date=1980-01-01,2017-06-01&title_type=tv_series&user_rating=1.0,10&view=simpl"
response = requests.get(url)
print 'Status Code: ',response.status_code
html = response.text



Status Code:  200


In [470]:
#Getting the ids from the 3 search results' pages - each one with 250 itens.
import lxml.html as lh
doc = lh.parse("http://www.imdb.com/search/title?count=250&countries=us&languages=en&num_votes=10000,&release_date=1980-01-01,2017-06-01&title_type=tv_series&user_rating=1.0,10&view=simple&sort=alpha,asc")
movie_id = doc.xpath("//div[@class='col-title']/span[@class='lister-item-header']/span[2]/a/@href")
new_id1 = []
for i in movie_id:
    new_id1.append(i[7:16])
    
doc = lh.parse("http://www.imdb.com/search/title?count=250&countries=us&languages=en&num_votes=10000,&release_date=1980-01-01,2017-06-01&title_type=tv_series&user_rating=1.0,10&view=simple&sort=alpha,asc&page=2&ref_=adv_nxt")
movie_id2 = doc.xpath("//div[@class='col-title']/span[@class='lister-item-header']/span[2]/a/@href")
new_id2 = []
for i in movie_id2:
    new_id2.append(i[7:16])
    
doc = lh.parse("http://www.imdb.com/search/title?count=250&countries=us&languages=en&num_votes=10000,&release_date=1980-01-01,2017-06-01&title_type=tv_series&user_rating=1.0,10&view=simple&sort=alpha,asc&page=3&ref_=adv_nxt")
movie_id3 = doc.xpath("//div[@class='col-title']/span[@class='lister-item-header']/span[2]/a/@href")
new_id3 = []
for i in movie_id3:
    new_id3.append(i[7:16])


In [42]:
#Creating id's list to parse to the urls to scrape details from each title in the list
id_list = new_id1 + new_id2 + new_id3
len(id_list)

615

In [472]:
#Preparing urls for each title to be used to scrape data via Selector / X-path
url = 'http://www.imdb.com/title/'

def newurl(id_list, url):
    html_list = []
    for x in id_list:
        url2 = url + x
        html_list.append(url2)
    return html_list
            
new_list = newurl(id_list, url)

urlf = '/?ref_=adv_li_tt'
final = [i + urlf for i in new_list]
final[0:5]


['http://www.imdb.com/title/tt3148266/?ref_=adv_li_tt',
 'http://www.imdb.com/title/tt1837492/?ref_=adv_li_tt',
 'http://www.imdb.com/title/tt1845307/?ref_=adv_li_tt',
 'http://www.imdb.com/title/tt0285331/?ref_=adv_li_tt',
 'http://www.imdb.com/title/tt0496424/?ref_=adv_li_tt']

In [79]:
#to avoid slowing down the kernel too much, I decided to break the final url's list into 4 
# passing each one of the urls to get the response as html - for each title to be scraped.
url_lista = []
for i in final[0:100]:
    response = requests.get(i)
    html = response.text
    url_lista.append(html)
    
url_listb = []
for i in final[100:300]:
    response = requests.get(i)
    html = response.text
    url_listb.append(html)
    
url_listc = []
for i in final[300:500]:
    response = requests.get(i)
    html = response.text
    url_listc.append(html)
    
url_listd = []
for i in final[500:616]:
    response = requests.get(i)
    html = response.text
    url_listd.append(html)


In [85]:
#Merging the html / response lists into one to be used in the for loops
#to scrape all titles at once for each info item needed
url_liste = url_lista + url_listb + url_listc + url_listd
len(url_liste)

615

In [230]:
#Getting the titles via Selector / X-path by iterating over each html
title = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/h1/text()''').extract()
    title.append(a)
    
title 
 

[[u'12 Monkeys\xa0            '],
 [u'13 Reasons Why\xa0            '],
 [u'2 Broke Girls\xa0            '],
 [u'24\xa0            '],
 [u'30 Rock\xa0            '],
 [u'3rd Rock from the Sun\xa0            '],
 [u'666 Park Avenue\xa0            '],
 [u'7th Heaven\xa0            '],
 [u'8 Simple Rules\xa0            '],
 [u'90210\xa0            '],
 [u'A Series of Unfortunate Events\xa0            '],
 [u'ALF\xa0            '],
 [u'According to Jim\xa0            '],
 [u'Adventure Time\xa0            '],
 [u'Agent Carter\xa0            '],
 [u'Agents of S.H.I.E.L.D.\xa0            '],
 [u'Alcatraz\xa0            '],
 [u'Alias\xa0            '],
 [u'Ally McBeal\xa0            '],
 [u'Almost Human\xa0            '],
 [u'Alphas\xa0            '],
 [u"America's Next Top Model\xa0            "],
 [u'American Crime Story\xa0            '],
 [u'American Dad!\xa0            '],
 [u'American Gods\xa0            '],
 [u'American Horror Story\xa0            '],
 [u'American Idol\xa0            ']

In [231]:
#Getting genre via Selector / X-path by iterating over each html
genre = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/div/a[1]/span/text()''').extract()
    genre.append(a)
    
genre
 

[[u'Adventure'],
 [u'Drama'],
 [u'Comedy'],
 [u'Action'],
 [u'Comedy'],
 [u'Comedy'],
 [u'Drama'],
 [u'Drama'],
 [u'Comedy'],
 [u'Comedy'],
 [u'Adventure'],
 [u'Comedy'],
 [u'Comedy'],
 [u'Animation'],
 [u'Action'],
 [u'Action'],
 [u'Action'],
 [u'Action'],
 [u'Comedy'],
 [u'Action'],
 [u'Action'],
 [u'Game-Show'],
 [u'Biography'],
 [u'Animation'],
 [u'Fantasy'],
 [u'Drama'],
 [u'Game-Show'],
 [u'Action'],
 [u'Comedy'],
 [u'Animation'],
 [u'Animation'],
 [u'Animation'],
 [u'Comedy'],
 [u'Action'],
 [u'Comedy'],
 [u'Animation'],
 [u'Drama'],
 [u'Comedy'],
 [u'Comedy'],
 [u'Action'],
 [u'Comedy'],
 [u'Action'],
 [u'Drama'],
 [u'Adventure'],
 [u'Animation'],
 [u'Action'],
 [u'Action'],
 [u'Animation'],
 [u'Drama'],
 [u'Drama'],
 [u'Crime'],
 [u'Comedy'],
 [u'Drama'],
 [u'Drama'],
 [u'Drama'],
 [u'Adventure'],
 [u'Comedy'],
 [u'Action'],
 [u'Drama'],
 [u'Crime'],
 [u'Comedy'],
 [u'Animation'],
 [u'Crime'],
 [u'Animation'],
 [u'Crime'],
 [u'Comedy'],
 [u'Comedy'],
 [u'Crime'],
 [u'Crime'],


In [232]:
#Getting creator via Selector / X-path by iterating over each html
creator = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[1]/div[2]/span[1]/a/span/text()''').extract()
    creator.append(a)
    
creator

[[u'Travis Fickett'],
 [u'Brian Yorkey'],
 [u'Whitney Cummings'],
 [u'Robert Cochran'],
 [],
 [],
 [u'David Wilcox'],
 [],
 [],
 [],
 [u'Neil Patrick Harris'],
 [],
 [],
 [u'Pendleton Ward'],
 [u'Christopher Markus'],
 [u'Maurissa Tancharoen'],
 [],
 [],
 [],
 [u'J.H. Wyman'],
 [u'Michael Karnow'],
 [],
 [u'Scott Alexander'],
 [],
 [u'Bryan Fuller'],
 [u'Brad Falchuk'],
 [u'Simon Fuller'],
 [],
 [u'Bruce Helford'],
 [],
 [],
 [u'Adam Reed'],
 [u'Mitchell Hurwitz'],
 [u'Marc Guggenheim'],
 [u'Donald Glover'],
 [],
 [],
 [u'Lauren Iungerich'],
 [],
 [],
 [u'Stephen Levinson'],
 [u'David Schickler'],
 [u'Anthony Cipriano'],
 [],
 [],
 [u'Glen A. Larson'],
 [],
 [],
 [],
 [u'Alfonso Cuar\xf3n'],
 [u'Vince Gilligan'],
 [],
 [],
 [u'Mark V. Olsen'],
 [u'Brian Koppelman'],
 [u'Robert Levine'],
 [u'Kenya Barris'],
 [u'Martin Gero'],
 [u'Glenn Kessler'],
 [],
 [],
 [u'Raphael Bob-Waksberg'],
 [u'Terence Winter'],
 [u'Loren Bouchard'],
 [],
 [u'Hart Hanson'],
 [],
 [u'Michael Connelly'],
 [u'Far

In [233]:
#Getting missing creators 1 via Selector / X-path by iterating over each html
creator1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[1]/div[2]/span/a/span/text()''').extract()
    creator1.append(a)
    
creator1

[[],
 [],
 [],
 [],
 [u'Tina Fey'],
 [u'Bonnie Turner', u'Terry Turner'],
 [],
 [u'Brenda Hampton'],
 [u'Tracy Gamble'],
 [u'Darren Star', u'Jeff Judah', u'Gabe Sachs'],
 [],
 [u'Tom Patchett', u'Paul Fusco'],
 [u'Tracy Newman', u'Jonathan Stark'],
 [],
 [],
 [],
 [u'Steven Lilien', u'Elizabeth Sarnoff', u'Bryan Wynbrandt'],
 [u'J.J. Abrams'],
 [u'David E. Kelley'],
 [],
 [],
 [u'Ken Mok', u'Tyra Banks', u'Kenya Barris'],
 [],
 [u'Mike Barker', u'Seth MacFarlane', u'Matt Weitzman'],
 [],
 [],
 [],
 [u'David Greenwalt', u'Joss Whedon'],
 [],
 [u'Tom Ruegger'],
 [u'Dave Willis', u'Matt Maiellaro'],
 [],
 [],
 [],
 [],
 [u'Michael Dante DiMartino', u'Bryan Konietzko'],
 [u'Kyle Killen'],
 [],
 [u'Dan Berendsen'],
 [u'J. Michael Straczynski'],
 [],
 [],
 [],
 [u'Bill Finger', u'Bob Kane'],
 [u'Bill Finger', u'Bob Kane', u'Eric Radomski'],
 [],
 [u'Michael Berk', u'Gregory J. Bonann', u'Douglas Schwartz'],
 [u'Mike Judge'],
 [u'Jeremy Carver', u'Anna Fricke'],
 [],
 [],
 [u'Victor Fresco'],

In [234]:
#Merging to creators list into one creators final
creatorf = [m+n for m,n in zip(creator,creator1)]
creatorf


[[u'Travis Fickett'],
 [u'Brian Yorkey'],
 [u'Whitney Cummings'],
 [u'Robert Cochran'],
 [u'Tina Fey'],
 [u'Bonnie Turner', u'Terry Turner'],
 [u'David Wilcox'],
 [u'Brenda Hampton'],
 [u'Tracy Gamble'],
 [u'Darren Star', u'Jeff Judah', u'Gabe Sachs'],
 [u'Neil Patrick Harris'],
 [u'Tom Patchett', u'Paul Fusco'],
 [u'Tracy Newman', u'Jonathan Stark'],
 [u'Pendleton Ward'],
 [u'Christopher Markus'],
 [u'Maurissa Tancharoen'],
 [u'Steven Lilien', u'Elizabeth Sarnoff', u'Bryan Wynbrandt'],
 [u'J.J. Abrams'],
 [u'David E. Kelley'],
 [u'J.H. Wyman'],
 [u'Michael Karnow'],
 [u'Ken Mok', u'Tyra Banks', u'Kenya Barris'],
 [u'Scott Alexander'],
 [u'Mike Barker', u'Seth MacFarlane', u'Matt Weitzman'],
 [u'Bryan Fuller'],
 [u'Brad Falchuk'],
 [u'Simon Fuller'],
 [u'David Greenwalt', u'Joss Whedon'],
 [u'Bruce Helford'],
 [u'Tom Ruegger'],
 [u'Dave Willis', u'Matt Maiellaro'],
 [u'Adam Reed'],
 [u'Mitchell Hurwitz'],
 [u'Marc Guggenheim'],
 [u'Donald Glover'],
 [u'Michael Dante DiMartino', u'Bryan

In [235]:
#Getting cast via Selector / X-path by iterating over each html
cast1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[1]/div[3]/span[1]/a/span/text()''').extract()
    cast1.append(a)
    
cast1

[[],
 [],
 [],
 [],
 [u'Tina Fey'],
 [u'John Lithgow'],
 [],
 [u'Stephen Collins'],
 [u'Katey Sagal'],
 [u'Shenae Grimes-Beech'],
 [],
 [u"Mihaly 'Michu' Meszaros"],
 [u'James Belushi'],
 [],
 [],
 [],
 [u'Sarah Jones'],
 [u'Jennifer Garner'],
 [u'Calista Flockhart'],
 [],
 [],
 [u'Adrianne Curry'],
 [],
 [u'Seth MacFarlane'],
 [],
 [],
 [],
 [u'David Boreanaz'],
 [],
 [u'Rob Paulsen'],
 [u'Dave Willis'],
 [],
 [],
 [],
 [],
 [u'Dee Bradley Baker'],
 [u'Jason Isaacs'],
 [],
 [u'Jean-Luc Bilodeau'],
 [u'Mira Furlan'],
 [],
 [],
 [],
 [u'Will Friedle'],
 [u'Kevin Conroy'],
 [],
 [u'David Hasselhoff'],
 [u'Mike Judge'],
 [u'Sam Witwer'],
 [],
 [],
 [u'Jay Harrington'],
 [u'Jason Priestley'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Donnie Wahlberg'],
 [u'Darin Brooks'],
 [],
 [],
 [],
 [u'Dana Delany'],
 [],
 [u'Jason Schwartzman'],
 [],
 [],
 [],
 [u'Ben Savage'],
 [],
 [u'Domenick Lombardozzi'],
 [],
 [],
 [u'Dave Annable'],
 [u'Sarah Michelle Gellar'],
 [u'Jeffrey Donovan'],
 [u'Laurence Fish

In [236]:
#Getting missing cast via Selector / X-path by iterating over each html
cast2 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[1]/div[3]/span[1]/a/span/text()''').extract()
    cast2.append(a)
    
cast2

[[u'Aaron Stanford'],
 [u'Dylan Minnette'],
 [u'Kat Dennings'],
 [u'Kiefer Sutherland'],
 [],
 [],
 [u'Rachael Taylor'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'John DiMaggio'],
 [u'Hayley Atwell'],
 [u'Clark Gregg'],
 [],
 [],
 [],
 [u'Karl Urban'],
 [u'David Strathairn'],
 [],
 [u'Cuba Gooding Jr.'],
 [],
 [u'Ricky Whittle'],
 [u'Lady Gaga'],
 [u'Ryan Seacrest'],
 [],
 [u'Shawnee Smith'],
 [],
 [],
 [u'H. Jon Benjamin'],
 [u'Jason Bateman'],
 [u'Stephen Amell'],
 [u'Donald Glover'],
 [],
 [],
 [u'Ashley Rickards'],
 [],
 [],
 [u'Dwayne Johnson'],
 [u'Antony Starr'],
 [u'Vera Farmiga'],
 [],
 [],
 [u'Edward James Olmos'],
 [],
 [],
 [],
 [u'Jake McLaughlin'],
 [u'Bob Odenkirk'],
 [],
 [],
 [u'Bill Paxton'],
 [u'Paul Giamatti'],
 [u'Jessica Parker Kennedy'],
 [u'Anthony Anderson'],
 [u'Sullivan Stapleton'],
 [u'Kyle Chandler'],
 [],
 [],
 [u'Will Arnett'],
 [u'Steve Buscemi'],
 [u'H. Jon Benjamin'],
 [],
 [u'Emily Deschanel'],
 [],
 [u'Titus Welliver'],
 [u'Kelsey Grammer'],
 [u'James Spader

In [237]:
#Getting missing cast via Selector / X-path by iterating over each html
cast3 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[1]/div[2]/span[1]/a/span/text()''').extract()
    cast3.append(a)
    
cast3

[[u'Travis Fickett'],
 [u'Brian Yorkey'],
 [u'Whitney Cummings'],
 [u'Robert Cochran'],
 [],
 [],
 [u'David Wilcox'],
 [],
 [],
 [],
 [u'Neil Patrick Harris'],
 [],
 [],
 [u'Pendleton Ward'],
 [u'Christopher Markus'],
 [u'Maurissa Tancharoen'],
 [],
 [],
 [],
 [u'J.H. Wyman'],
 [u'Michael Karnow'],
 [],
 [u'Scott Alexander'],
 [],
 [u'Bryan Fuller'],
 [u'Brad Falchuk'],
 [u'Simon Fuller'],
 [],
 [u'Bruce Helford'],
 [],
 [],
 [u'Adam Reed'],
 [u'Mitchell Hurwitz'],
 [u'Marc Guggenheim'],
 [u'Donald Glover'],
 [],
 [],
 [u'Lauren Iungerich'],
 [],
 [],
 [u'Stephen Levinson'],
 [u'David Schickler'],
 [u'Anthony Cipriano'],
 [],
 [],
 [u'Glen A. Larson'],
 [],
 [],
 [],
 [u'Alfonso Cuar\xf3n'],
 [u'Vince Gilligan'],
 [],
 [],
 [u'Mark V. Olsen'],
 [u'Brian Koppelman'],
 [u'Robert Levine'],
 [u'Kenya Barris'],
 [u'Martin Gero'],
 [u'Glenn Kessler'],
 [],
 [],
 [u'Raphael Bob-Waksberg'],
 [u'Terence Winter'],
 [u'Loren Bouchard'],
 [],
 [u'Hart Hanson'],
 [],
 [u'Michael Connelly'],
 [u'Far

In [238]:
#Merging cast info into a final cast list
cast = [m+n+o for m,n,o in zip(cast1,cast2,cast3)]
cast

[[u'Aaron Stanford', u'Travis Fickett'],
 [u'Dylan Minnette', u'Brian Yorkey'],
 [u'Kat Dennings', u'Whitney Cummings'],
 [u'Kiefer Sutherland', u'Robert Cochran'],
 [u'Tina Fey'],
 [u'John Lithgow'],
 [u'Rachael Taylor', u'David Wilcox'],
 [u'Stephen Collins'],
 [u'Katey Sagal'],
 [u'Shenae Grimes-Beech'],
 [u'Neil Patrick Harris'],
 [u"Mihaly 'Michu' Meszaros"],
 [u'James Belushi'],
 [u'John DiMaggio', u'Pendleton Ward'],
 [u'Hayley Atwell', u'Christopher Markus'],
 [u'Clark Gregg', u'Maurissa Tancharoen'],
 [u'Sarah Jones'],
 [u'Jennifer Garner'],
 [u'Calista Flockhart'],
 [u'Karl Urban', u'J.H. Wyman'],
 [u'David Strathairn', u'Michael Karnow'],
 [u'Adrianne Curry'],
 [u'Cuba Gooding Jr.', u'Scott Alexander'],
 [u'Seth MacFarlane'],
 [u'Ricky Whittle', u'Bryan Fuller'],
 [u'Lady Gaga', u'Brad Falchuk'],
 [u'Ryan Seacrest', u'Simon Fuller'],
 [u'David Boreanaz'],
 [u'Shawnee Smith', u'Bruce Helford'],
 [u'Rob Paulsen'],
 [u'Dave Willis'],
 [u'H. Jon Benjamin', u'Adam Reed'],
 [u'Jas

In [239]:
#Getting rate via Selector / X-path by iterating over each html
rate = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[1]/div[1]/div[1]/strong/span/text()''').extract()
    rate.append(a)
    
rate

[[u'7.6'],
 [u'8.5'],
 [u'6.7'],
 [u'8.4'],
 [u'8.2'],
 [u'7.8'],
 [u'6.9'],
 [u'5.1'],
 [u'7.1'],
 [u'6.1'],
 [u'7.9'],
 [u'7.4'],
 [u'6.4'],
 [u'8.6'],
 [u'8.0'],
 [u'7.5'],
 [u'7.0'],
 [u'7.6'],
 [u'6.8'],
 [u'8.1'],
 [u'7.3'],
 [u'5.6'],
 [u'8.5'],
 [u'7.4'],
 [u'8.2'],
 [u'8.2'],
 [u'4.2'],
 [u'8.0'],
 [u'6.7'],
 [u'7.9'],
 [u'7.7'],
 [u'8.7'],
 [u'8.9'],
 [u'7.8'],
 [u'8.6'],
 [u'9.2'],
 [u'7.9'],
 [u'7.6'],
 [u'7.4'],
 [u'8.3'],
 [u'7.6'],
 [u'8.4'],
 [u'8.2'],
 [u'8.1'],
 [u'9.0'],
 [u'8.7'],
 [u'5.2'],
 [u'7.5'],
 [u'7.5'],
 [u'7.2'],
 [u'8.7'],
 [u'8.2'],
 [u'6.2'],
 [u'7.6'],
 [u'8.4'],
 [u'8.3'],
 [u'7.1'],
 [u'7.6'],
 [u'8.1'],
 [u'7.5'],
 [u'8.4'],
 [u'8.4'],
 [u'8.6'],
 [u'8.1'],
 [u'7.1'],
 [u'7.9'],
 [u'7.9'],
 [u'8.3'],
 [u'8.1'],
 [u'8.4'],
 [u'8.1'],
 [u'9.5'],
 [u'7.3'],
 [u'8.4'],
 [u'8.3'],
 [u'7.3'],
 [u'8.2'],
 [u'8.0'],
 [u'7.7'],
 [u'6.4'],
 [u'7.0'],
 [u'8.3'],
 [u'6.5'],
 [u'7.1'],
 [u'8.5'],
 [u'8.2'],
 [u'6.6'],
 [u'8.7'],
 [u'7.1'],
 [u'7.8'],
 [u'7.9'],

In [240]:
#Getting rateconunt (number of raters) via Selector / X-path by iterating over each html
ratecount = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[1]/div[1]/a/span/text()''').extract()
    ratecount.append(a)
    
ratecount

[[u'27,111'],
 [u'122,904'],
 [u'77,915'],
 [u'155,403'],
 [u'96,989'],
 [u'40,250'],
 [u'16,200'],
 [u'21,794'],
 [u'21,449'],
 [u'37,999'],
 [u'27,517'],
 [u'25,367'],
 [u'30,567'],
 [u'55,124'],
 [u'57,212'],
 [u'159,029'],
 [u'36,342'],
 [u'39,340'],
 [u'25,387'],
 [u'43,129'],
 [u'24,984'],
 [u'16,128'],
 [u'45,305'],
 [u'93,842'],
 [u'33,907'],
 [u'216,280'],
 [u'19,864'],
 [u'54,697'],
 [u'27,957'],
 [u'17,363'],
 [u'19,730'],
 [u'105,510'],
 [u'228,506'],
 [u'338,136'],
 [u'18,283'],
 [u'153,320'],
 [u'19,153'],
 [u'38,776'],
 [u'15,944'],
 [u'23,766'],
 [u'24,191'],
 [u'66,449'],
 [u'71,759'],
 [u'19,434'],
 [u'61,001'],
 [u'129,128'],
 [u'20,412'],
 [u'22,964'],
 [u'22,122'],
 [u'13,224'],
 [u'195,181'],
 [u'22,584'],
 [u'25,099'],
 [u'18,224'],
 [u'33,413'],
 [u'69,285'],
 [u'12,250'],
 [u'48,352'],
 [u'33,246'],
 [u'23,875'],
 [u'41,942'],
 [u'37,970'],
 [u'146,277'],
 [u'45,444'],
 [u'12,727'],
 [u'120,761'],
 [u'28,833'],
 [u'22,625'],
 [u'15,865'],
 [u'37,451'],
 [u'33,6

In [241]:
#Getting review (number of reviewers) via Selector / X-path by iterating over each html
review = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[1]/div[2]/span/a[1]/text()''').extract()
    review.append(a)
review

[[u'74 user'],
 [u'571 user'],
 [u'255 user'],
 [u'573 user'],
 [],
 [],
 [u'44 user'],
 [],
 [],
 [],
 [u'113 user'],
 [],
 [],
 [u'126 user'],
 [u'77 user'],
 [u'317 user'],
 [],
 [],
 [],
 [u'93 user'],
 [u'70 user'],
 [],
 [u'78 user'],
 [],
 [u'166 user'],
 [u'342 user'],
 [u'139 user'],
 [],
 [u'59 user'],
 [],
 [],
 [u'95 user'],
 [u'300 user'],
 [u'543 user'],
 [u'27 user'],
 [],
 [],
 [u'46 user'],
 [],
 [],
 [u'37 user'],
 [u'155 user'],
 [u'152 user'],
 [],
 [],
 [u'650 user'],
 [],
 [],
 [],
 [u'69 user'],
 [u'243 user'],
 [],
 [],
 [u'66 user'],
 [u'70 user'],
 [u'188 user'],
 [u'66 user'],
 [u'155 user'],
 [u'163 user'],
 [],
 [],
 [u'89 user'],
 [u'148 user'],
 [u'103 user'],
 [],
 [u'179 user'],
 [],
 [u'190 user'],
 [u'47 user'],
 [u'151 user'],
 [],
 [u'958 user'],
 [],
 [u'38 user'],
 [u'82 user'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'113 user'],
 [u'121 user'],
 [],
 [u'141 user'],
 [],
 [],
 [],
 [],
 [u'57 user'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'78 user'],
 [u

In [242]:
#Getting missing review (number of reviewers) via Selector / X-path by iterating over each html
user1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[2]/div[1]/div[2]/span/a[1]/text()''').extract()
    user1.append(a)
user1

[[],
 [],
 [],
 [],
 [u'89 user'],
 [u'89 user'],
 [],
 [u'270 user'],
 [u'119 user'],
 [u'91 user'],
 [],
 [u'51 user'],
 [u'58 user'],
 [],
 [],
 [],
 [u'75 user'],
 [u'215 user'],
 [u'123 user'],
 [],
 [],
 [u'42 user'],
 [],
 [u'107 user'],
 [],
 [],
 [],
 [u'202 user'],
 [],
 [u'62 user'],
 [u'71 user'],
 [],
 [],
 [],
 [],
 [u'278 user'],
 [u'62 user'],
 [],
 [u'29 user'],
 [u'174 user'],
 [],
 [],
 [],
 [u'68 user'],
 [u'113 user'],
 [],
 [u'49 user'],
 [u'98 user'],
 [u'81 user'],
 [],
 [],
 [u'39 user'],
 [u'63 user'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'104 user'],
 [u'36 user'],
 [],
 [],
 [],
 [u'45 user'],
 [],
 [u'30 user'],
 [],
 [],
 [],
 [u'257 user'],
 [],
 [u'30 user'],
 [],
 [],
 [u'100 user'],
 [u'679 user'],
 [u'154 user'],
 [u'235 user'],
 [u'238 user'],
 [u'78 user'],
 [u'179 user'],
 [],
 [],
 [u'115 user'],
 [],
 [u'34 user'],
 [u'87 user'],
 [u'357 user'],
 [u'104 user'],
 [],
 [u'41 user'],
 [u'22 user'],
 [u'322 user'],
 [u'87 user'],
 [u'34 user'],
 [u'80 us

In [243]:
#Merging into final reviews list (number of users who reviewed shows)
reviewf = [m+n for m,n in zip(review,user1)]
reviewf

[[u'74 user'],
 [u'571 user'],
 [u'255 user'],
 [u'573 user'],
 [u'89 user'],
 [u'89 user'],
 [u'44 user'],
 [u'270 user'],
 [u'119 user'],
 [u'91 user'],
 [u'113 user'],
 [u'51 user'],
 [u'58 user'],
 [u'126 user'],
 [u'77 user'],
 [u'317 user'],
 [u'75 user'],
 [u'215 user'],
 [u'123 user'],
 [u'93 user'],
 [u'70 user'],
 [u'42 user'],
 [u'78 user'],
 [u'107 user'],
 [u'166 user'],
 [u'342 user'],
 [u'139 user'],
 [u'202 user'],
 [u'59 user'],
 [u'62 user'],
 [u'71 user'],
 [u'95 user'],
 [u'300 user'],
 [u'543 user'],
 [u'27 user'],
 [u'278 user'],
 [u'62 user'],
 [u'46 user'],
 [u'29 user'],
 [u'174 user'],
 [u'37 user'],
 [u'155 user'],
 [u'152 user'],
 [u'68 user'],
 [u'113 user'],
 [u'650 user'],
 [u'49 user'],
 [u'98 user'],
 [u'81 user'],
 [u'69 user'],
 [u'243 user'],
 [u'39 user'],
 [u'63 user'],
 [u'66 user'],
 [u'70 user'],
 [u'188 user'],
 [u'66 user'],
 [u'155 user'],
 [u'163 user'],
 [u'104 user'],
 [u'36 user'],
 [u'89 user'],
 [u'148 user'],
 [u'103 user'],
 [u'45 use

In [244]:
#Getting popularity via Selector / X-path by iterating over each html
pop1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[3]/div[2]/div[2]/span/text()''').extract()
    pop1.append(a[0:1])
pop1

[[u'\n                    310\n      ('],
 [u'\n                    43\n      ('],
 [u'\n                    280\n      ('],
 [u'\n                    231\n      ('],
 [],
 [],
 [u'\n                    2,304\n      ('],
 [],
 [],
 [],
 [u'\n                    285\n      ('],
 [],
 [],
 [u'\n                    305\n      ('],
 [u'\n                    487\n      ('],
 [u'\n                    81\n                '],
 [],
 [],
 [],
 [u'\n                    1,393\n      ('],
 [u'\n                    1,522\n      ('],
 [],
 [u'\n                    284\n      ('],
 [],
 [u'\n                    63\n      ('],
 [u'\n                    15\n                '],
 [u'\n                    1,714\n      ('],
 [],
 [u'\n                    867\n      ('],
 [],
 [],
 [u'\n                    133\n      ('],
 [u'\n                    164\n      ('],
 [u'\n                    18\n      ('],
 [u'\n                    295\n      ('],
 [],
 [],
 [u'\n                    689\n      ('],
 [],
 [],
 [

In [245]:
#Getting missing popularity via Selector / X-path by iterating over each html
pop2 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[3]/div[2]/div[2]/div[3]/div[2]/div[2]/span/text()''').extract()
    pop2.append(a[0:1])
pop2

[[],
 [],
 [],
 [],
 [u'\n                    328\n      ('],
 [u'\n                    749\n      ('],
 [],
 [u'\n                    386\n      ('],
 [u'\n                    555\n      ('],
 [u'\n                    347\n      ('],
 [],
 [u'\n                    639\n      ('],
 [u'\n                    1,144\n      ('],
 [],
 [],
 [],
 [u'\n                    1,924\n      ('],
 [u'\n                    713\n      ('],
 [u'\n                    741\n      ('],
 [],
 [],
 [u'\n                    754\n      ('],
 [],
 [u'\n                    143\n      ('],
 [],
 [],
 [],
 [u'\n                    446\n      ('],
 [],
 [u'\n                    1,744\n      ('],
 [u'\n                    1,756\n      ('],
 [],
 [],
 [],
 [],
 [u'\n                    235\n      ('],
 [u'\n                    2,309\n      ('],
 [],
 [u'\n                    303\n      ('],
 [u'\n                    721\n      ('],
 [],
 [],
 [],
 [u'\n                    1,411\n      ('],
 [u'\n                    46

In [246]:
#Merging into final popularity list
pop = [m+n for m,n in zip(pop1,pop2)]
pop

[[u'\n                    310\n      ('],
 [u'\n                    43\n      ('],
 [u'\n                    280\n      ('],
 [u'\n                    231\n      ('],
 [u'\n                    328\n      ('],
 [u'\n                    749\n      ('],
 [u'\n                    2,304\n      ('],
 [u'\n                    386\n      ('],
 [u'\n                    555\n      ('],
 [u'\n                    347\n      ('],
 [u'\n                    285\n      ('],
 [u'\n                    639\n      ('],
 [u'\n                    1,144\n      ('],
 [u'\n                    305\n      ('],
 [u'\n                    487\n      ('],
 [u'\n                    81\n                '],
 [u'\n                    1,924\n      ('],
 [u'\n                    713\n      ('],
 [u'\n                    741\n      ('],
 [u'\n                    1,393\n      ('],
 [u'\n                    1,522\n      ('],
 [u'\n                    754\n      ('],
 [u'\n                    284\n      ('],
 [u'\n           

In [247]:
#Getting number of episodes via Selector / X-path by iterating over each html
episodes = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="title-overview-widget"]/div[2]/div[3]/a/div/div/span/text()''').extract()
    episodes.append(a)
episodes


[[u'46 episodes'],
 [u'21 episodes'],
 [u'137 episodes'],
 [u'195 episodes'],
 [u'138 episodes'],
 [u'139 episodes'],
 [u'13 episodes'],
 [u'243 episodes'],
 [u'76 episodes'],
 [u'114 episodes'],
 [u'26 episodes'],
 [u'103 episodes'],
 [u'182 episodes'],
 [u'275 episodes'],
 [u'18 episodes'],
 [u'110 episodes'],
 [u'13 episodes'],
 [u'105 episodes'],
 [u'112 episodes'],
 [u'13 episodes'],
 [u'24 episodes'],
 [u'308 episodes'],
 [u'31 episodes'],
 [u'236 episodes'],
 [u'16 episodes'],
 [u'86 episodes'],
 [u'578 episodes'],
 [u'111 episodes'],
 [u'100 episodes'],
 [u'99 episodes'],
 [u'140 episodes'],
 [u'98 episodes'],
 [u'85 episodes'],
 [u'139 episodes'],
 [u'11 episodes'],
 [u'66 episodes'],
 [u'13 episodes'],
 [u'89 episodes'],
 [u'100 episodes'],
 [u'110 episodes'],
 [u'31 episodes'],
 [u'38 episodes'],
 [u'50 episodes'],
 [u'52 episodes'],
 [u'85 episodes'],
 [u'73 episodes'],
 [u'242 episodes'],
 [u'208 episodes'],
 [u'52 episodes'],
 [u'13 episodes'],
 [u'40 episodes'],
 [u'26 e

In [248]:
#Getting country info via Selector / X-path by iterating over each html
country = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[2]/a/text()''').extract()
    country.append(a)
country

[[u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'English'],
 [u'USA'],
 [u'English'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'English',
  u'Bulgarian',
  u'Arabic',
  u'Cantonese',
  u'French',
  u'German',
  u'Hebrew',
  u'Hindi',
  u'Hungarian',
  u'Indonesian',
  u'Italian',
  u'Japanese',
  u'Portuguese',
  u'Romanian',
  u'Russian',
  u'Spanish',
  u'Thai',
  u'Ukrainian',
  u'Uzbek',
  u'Dutch'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'English'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'USA'],
 [u'English'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'English'],
 [u'USA'],
 [u'English'],
 [u'USA'],
 [u'USA'],
 [u'English'],
 [u'English'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'Canada', u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'USA'],
 [u'South Africa', u'USA'],
 [u'English'],
 [u'USA'],
 [u'U

In [249]:
#Getting release date via Selector / X-path by iterating over each html
date = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[4]/text()''').extract()
    date.append(a)
date

[[u'\n    ', u' 16 January 2015 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 31 March 2017 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 19 September 2011 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 6 November 2001 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 11 October 2006 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 9 January 1996 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 2012 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 26 August 1996 (USA)\n    ', u'\n    '],
 [u'\n      ', u' 8 Simple Rules\n      ', u'\n      '],
 [u'\n      ', u' 90210: The Next Generation\n      ', u'\n      '],
 [u'\n    ', u' 13 January 2017 (USA)\n    ', u'\n    '],
 [u'\n      ', u' ALF, o Eteimoso\n      ', u'\n      '],
 [u'\n    ', u' 3 October 2001 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 5 April 2010 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 6 January 2015 (USA)\n    ', u'\n    '],
 [u'\n    ', u' 24 September 2013 (USA)\n    ', u'\n    '],
 [u'\n      ', u' Alkatraz\n      ', u'\n      '],
 [u'\n      ', u' A Vingado

In [250]:
#Getting language via Selector / X-path by iterating over each html
language = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[3]/a/text()''').extract()
    language.append(a)
language

[[u'English'],
 [u'English'],
 [u'English',
  u'Korean',
  u'Yiddish',
  u'Hebrew',
  u'Spanish',
  u'Arabic',
  u'Ukrainian'],
 [u'English',
  u'Russian',
  u'German',
  u'Korean',
  u'Arabic',
  u'Spanish',
  u'Serbian',
  u'Mandarin'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [],
 [u'English'],
 [u'English', u'Korean'],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [u'English'],
 [u'English', u'Persian'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [u'English'],
 [],
 [u'English'],
 [],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English', u'Spanish', u'Vietnamese'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [u'English'],
 [],
 [u'English'],
 [u'English'],
 [],
 [],
 [u'English'],
 [u'English

In [251]:
#Getting production's info via Selector / X-path by iterating over each html
production = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[7]/span[2]/a/span/text()''').extract()
    production.append(a)
production

[[],
 [u'Kicked to the Curb Productions'],
 [u'Warner Bros. Television'],
 [],
 [u'Little Stranger'],
 [u'YBYL Productions'],
 [u'Alloy Entertainment'],
 [],
 [],
 [],
 [u'What is the Question?'],
 [],
 [u'Suzanne Bukinik Entertainment'],
 [],
 [u'Marvel Television'],
 [u'Disney-ABC Domestic Television'],
 [],
 [],
 [u'David E. Kelley Productions'],
 [u'Bad Robot'],
 [u'Universal Cable Productions'],
 [u'Anisa Productions'],
 [u'Color Force'],
 [],
 [u'Fremantle Media North America'],
 [u'Ryan Murphy Productions'],
 [u'Fremantle Media'],
 [],
 [],
 [],
 [],
 [u'FX Productions'],
 [],
 [u'DC Entertainment'],
 [],
 [],
 [u'Teakwood Lane Productions'],
 [],
 [],
 [],
 [],
 [u'Cinemax'],
 [],
 [],
 [],
 [u'David Eick Productions'],
 [],
 [],
 [u'Zodiak USA'],
 [u'Bonanza Productions'],
 [u'Crystal Diner Productions'],
 [u'20th Century Fox Television'],
 [u'Fair Dinkum Productions'],
 [u'Playtone Productions'],
 [],
 [u'Platinum Dunes'],
 [],
 [u'Berlanti Productions'],
 [u'Sony Pictures Te

In [252]:
#Getting missing production's info via Selector / X-path by iterating over each html
production1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[7]/span[1]/a/span/text()''').extract()
    production1.append(a)
production1


[[u'Atlas Entertainment'],
 [u'July Moon Productions'],
 [u'Michael Patrick King Productions'],
 [],
 [u'Broadway Video'],
 [u'Carsey-Werner Company'],
 [u'Bonanza Productions'],
 [u'Spelling Television'],
 [],
 [],
 [u'Sonnenfeld Company, The'],
 [],
 [u'Newman/Stark'],
 [],
 [u'Fazekas & Butters'],
 [u'ABC Studios'],
 [],
 [],
 [u'20th Century Fox Television'],
 [u'Frequency Films'],
 [u'BermanBraun'],
 [u'10 by 10 Entertainment'],
 [u'Ryan Murphy Productions'],
 [],
 [u'Canada Film Capital'],
 [u'Brad Falchuk Teley-Vision'],
 [u'Fremantle Media North America'],
 [],
 [],
 [],
 [],
 [u'Floyd County Productions'],
 [],
 [u'Berlanti Productions'],
 [],
 [],
 [u'Letter Eleven'],
 [],
 [],
 [],
 [],
 [u'Your Face Goes Here Entertainment'],
 [],
 [],
 [],
 [u'British Sky Broadcasting (BSkyB)'],
 [],
 [],
 [u'Muse Entertainment Enterprises'],
 [u'Bad Robot'],
 [u'High Bridge Productions'],
 [u'Garfield Grove Productions'],
 [u'90210 Productions'],
 [u'Anima Sola Productions'],
 [u'Showtime

In [253]:
#Getting missing production's info via Selector / X-path by iterating over each html
prod2 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[8]/span[2]/a/span/text()''').extract()
    prod2.append(a)
prod2

[[],
 [],
 [],
 [u'20th Century Fox Television'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Greenwolf'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Warner Bros. Television'],
 [],
 [],
 [u'Kerry Ehrin Productions'],
 [],
 [],
 [],
 [u'Pearson All-American Television'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Kuzui Enterprises'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'DeKnight Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Cherry Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'3 Arts Entertainment'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [254]:
#Getting missing production's info via Selector / X-path by iterating over each html
prod3 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[6]/span[3]/a/span/text()''').extract()
    prod3.append(a)
prod3

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Flody Co.'],
 [u'CBS Television Studios'],
 [],
 [],
 [],
 [u'Cartoon Network'],
 [],
 [],
 [u'Warner Bros. Television'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Fox Television Animation'],
 [],
 [],
 [],
 [],
 [u'Estevez Sheen Productions'],
 [],
 [u'Radical Axis'],
 [],
 [u'Hurwitz Company, The'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Leverage Management'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Judgemental Films Inc.'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Cinema Gypsy Productions'],
 [],
 [],
 [u'CBS Productions'],
 [u'Varsity Pictures'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Amazon Studios'],
 [u'Old Friends Productions'],
 [],
 [],
 [],
 [u'Chernin Entertainment'],
 [u'Paper Kite Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'KA Television Productions'],
 [],
 [],
 [],
 [],
 [],
 [u'Paramount Pictures'],
 [],
 [],
 [],
 [],
 [],
 [u'Touchstone Television'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'New Line Television'],
 []

In [255]:
#Getting missing production's info via Selector / X-path by iterating over each html
prod4 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[5]/span[2]/a/span/text()''').extract()
    prod4.append(a)
prod4

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Amblin Entertainment'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'DC Comics'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Wilo Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Nicktoons Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Nickelodeon Animation Studios'],
 [],
 [u'MTV Networks'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Gaumont Animation'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'D

In [272]:
#Getting missing production's info via Selector / X-path by iterating over each html
prod5 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[6]/span[1]/a/span/text()''').extract()
    prod5.append(a)
prod5

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Touchstone Television'],
 [u'Sachs/Judah Productions'],
 [],
 [u'Alien Productions'],
 [],
 [u'Frederator Studios'],
 [],
 [],
 [u'Bonanza Productions'],
 [u'Touchstone Television'],
 [],
 [],
 [],
 [],
 [],
 [u'20th Century Fox Television'],
 [],
 [],
 [],
 [],
 [u'Mohawk Productions'],
 [],
 [u'Awesome Incorporated'],
 [],
 [u'Imagine Entertainment'],
 [],
 [u'FX Productions'],
 [u'Nickelodeon Animation Studios'],
 [],
 [u'MTV/Remote Productions'],
 [u"Don't Borrow Trouble"],
 [],
 [u'Closest to the Hole Productions'],
 [],
 [],
 [],
 [u'Warner Bros. Animation'],
 [],
 [],
 [u'Film Roman Productions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'Wilmore Films'],
 [],
 [],
 [u'Panda Productions Inc.'],
 [u'Blue Mountain State Productions'],
 [u'Tornante Company'],
 [],
 [],
 [],
 [],
 [u'Dakota Pictures'],
 [u'Hieronymus Pictures'],
 [u'Grammnet Productions'],
 [],
 [],
 [],
 [u'Matt Olmstead Productions'],
 [u'3 Arts Entertainment'],
 [],
 

In [273]:
#Merging into final's production info list
prod = [m+n+o+p+q for m,n,o,p,q in zip(production1,prod2,prod3,prod4,prod5)]
prod

[[u'Atlas Entertainment'],
 [u'July Moon Productions'],
 [u'Michael Patrick King Productions'],
 [u'20th Century Fox Television'],
 [u'Broadway Video'],
 [u'Carsey-Werner Company'],
 [u'Bonanza Productions'],
 [u'Spelling Television'],
 [u'Flody Co.', u'Touchstone Television'],
 [u'CBS Television Studios', u'Sachs/Judah Productions'],
 [u'Sonnenfeld Company, The'],
 [u'Alien Productions'],
 [u'Newman/Stark'],
 [u'Cartoon Network', u'Frederator Studios'],
 [u'Fazekas & Butters'],
 [u'ABC Studios'],
 [u'Warner Bros. Television', u'Bonanza Productions'],
 [u'Touchstone Television'],
 [u'20th Century Fox Television'],
 [u'Frequency Films'],
 [u'BermanBraun'],
 [u'10 by 10 Entertainment'],
 [u'Ryan Murphy Productions'],
 [u'Fox Television Animation', u'20th Century Fox Television'],
 [u'Canada Film Capital'],
 [u'Brad Falchuk Teley-Vision'],
 [u'Fremantle Media North America'],
 [u'Greenwolf'],
 [u'Estevez Sheen Productions', u'Mohawk Productions'],
 [u'Amblin Entertainment'],
 [u'Radical A

In [257]:
#Getting runtime via Selector / X-path by iterating over each html
runtime1 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[9]/time/text()''').extract()
    runtime1.append(a)
runtime1

[[u'42 min'],
 [u'60 min'],
 [u'22 min'],
 [],
 [u'22 min'],
 [u'25 min', u'60 min'],
 [u'60 min'],
 [u'60 min'],
 [],
 [],
 [u'50 min'],
 [],
 [u'30 min'],
 [],
 [u'42 min', u'757 min', u'333 min', u'424 min'],
 [u'45 min'],
 [],
 [],
 [u'45 min', u'60 min'],
 [u'60 min'],
 [u'44 min'],
 [u'42 min'],
 [u'42 min'],
 [],
 [u'60 min'],
 [u'60 min'],
 [u'42 min'],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [u'42 min'],
 [],
 [],
 [u'43 min'],
 [],
 [],
 [],
 [],
 [u'60 min'],
 [],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [u'44 min'],
 [u'43 min'],
 [u'46 min'],
 [],
 [u'44 min', u'60 min', u'60 min'],
 [u'60 min'],
 [u'60 min'],
 [u'56 min'],
 [],
 [u'42 min'],
 [u'60 min'],
 [],
 [],
 [],
 [u'55 min'],
 [],
 [],
 [u'40 min'],
 [],
 [],
 [],
 [u'44 min'],
 [u'23 min'],
 [u'49 min', u'3,030 min'],
 [],
 [],
 [u'22 min'],
 [],
 [],
 [u'44 min'],
 [u'60 min'],
 [u'43 min'],
 [],
 [u'28 min'],
 [],
 [],
 [u'55 min'],
 [u'43 min'],
 [],
 [u'22 min'],
 [],
 [],
 [],
 [u'43 min'],
 [],
 [u'43 min'],
 [],

In [258]:
#Getting missing runtime via Selector / X-path by iterating over each html
runtime2 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[10]/time/text()''').extract()
    runtime2.append(a)
runtime2

[[],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [],
 [u'60 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'54 min', u'709 min', u'707 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'50 min', u'60 min', u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],

In [259]:
#Getting missing runtime via Selector / X-path by iterating over each html
runtime3 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[8]/time/text()''').extract()
    runtime3.append(a)
runtime3

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'30 min'],
 [],
 [],
 [u'25 min', u'30 min'],
 [],
 [u'11 min'],
 [],
 [],
 [u'60 min'],
 [u'42 min'],
 [],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [u'15 min'],
 [],
 [u'22 min'],
 [],
 [u'30 min'],
 [u'23 min'],
 [],
 [u'22 min'],
 [u'22 min'],
 [],
 [u'30 min'],
 [],
 [],
 [],
 [u'23 min'],
 [],
 [],
 [u'15 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [],
 [],
 [u'22 min'],
 [u'25 min'],
 [],
 [],
 [],
 [],
 [u'28 min'],
 [u'51 min'],
 [u'56 min'],
 [],
 [],
 [],
 [u'43 min'],
 [u'22 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [],
 [],
 [],
 [u'42 min'],
 [u'22 min', u'30 min', u'98 min'],
 [u'43 min'],
 [],
 [u'22 min'],
 [],
 [u'22 min', u'22 min', u'130 min', u'143 min'],
 [u'30 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'11 min'],
 [],
 [u'22 min', u'11 min'],
 [],
 [],
 [],
 [],
 [u'60 min'],
 [],
 [],
 [],
 [],
 [u'30 min'],
 [],
 [],
 [],
 [u'30 min'],
 [

In [309]:
#Getting missing runtime via Selector / X-path by iterating over each html
runtime4 = []
for i in url_liste:
    a = Selector(text=i).xpath('''//*[@id="titleDetails"]/div[10]/time/text()''').extract()
    runtime4.append(a)
runtime4

[[],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [],
 [u'60 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'54 min', u'709 min', u'707 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'45 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'22 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [u'50 min', u'60 min', u'44 min'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],

In [311]:
#Merging into final runtime list
runtime = [m+n+o for m,n,o in zip(runtime1,runtime2, runtime3)]
runtime

[[u'42 min'],
 [u'60 min'],
 [u'22 min'],
 [u'44 min'],
 [u'22 min'],
 [u'25 min', u'60 min'],
 [u'60 min'],
 [u'60 min'],
 [u'30 min'],
 [],
 [u'50 min'],
 [u'25 min', u'30 min'],
 [u'30 min'],
 [u'11 min'],
 [u'42 min', u'757 min', u'333 min', u'424 min'],
 [u'45 min'],
 [u'60 min'],
 [u'42 min'],
 [u'45 min', u'60 min'],
 [u'60 min'],
 [u'44 min'],
 [u'42 min'],
 [u'42 min'],
 [u'22 min'],
 [u'60 min'],
 [u'60 min'],
 [u'42 min'],
 [u'44 min'],
 [u'22 min'],
 [],
 [u'15 min'],
 [u'22 min'],
 [u'22 min'],
 [u'42 min'],
 [u'30 min'],
 [u'23 min'],
 [u'43 min'],
 [u'22 min'],
 [u'22 min'],
 [u'45 min'],
 [u'30 min'],
 [u'60 min'],
 [u'45 min'],
 [],
 [u'23 min'],
 [u'44 min'],
 [u'60 min'],
 [u'15 min'],
 [u'44 min'],
 [u'43 min'],
 [u'46 min'],
 [],
 [u'44 min', u'60 min', u'60 min'],
 [u'60 min'],
 [u'60 min'],
 [u'56 min'],
 [u'22 min'],
 [u'42 min'],
 [u'60 min'],
 [],
 [u'22 min'],
 [u'25 min'],
 [u'55 min'],
 [],
 [],
 [u'40 min'],
 [u'28 min'],
 [u'51 min'],
 [u'56 min'],
 [u'44

In [440]:
#Creating a data frame with all scraped data:
import pandas as pd
df = pd.DataFrame({'titles': title, 'genre': genre, 'creator': creatorf, 'cast': cast, \
                   'rate': rate, 'ratecount': ratecount, 'runtime':runtime, \
                   'production':prod, 'language':language, 'date':date, \
                   'country':country, 'popularity': pop, 'reviews':reviewf, \
                  'episodes': episodes})


In [441]:
df

,cast,country,creator,date,episodes,genre,language,popularity,production,rate,ratecount,reviews,runtime,titles
0,"[Aaron Stanford, Travis Fickett]",[USA],[Travis Fickett],"[\n , 16 January 2015 (USA)\n , \n ]",[46 episodes],[Adventure],[English],[\n 310\n (],[Atlas Entertainment],[7.6],"[27,111]",[74 user],[42 min],[12 Monkeys ]
1,"[Dylan Minnette, Brian Yorkey]",[USA],[Brian Yorkey],"[\n , 31 March 2017 (USA)\n , \n ]",[21 episodes],[Drama],[English],[\n 43\n (],[July Moon Productions],[8.5],"[122,904]",[571 user],[60 min],[13 Reasons Why ]
2,"[Kat Dennings, Whitney Cummings]",[USA],[Whitney Cummings],"[\n , 19 September 2011 (USA)\n , \n ]",[137 episodes],[Comedy],"[English, Korean, Yiddish, Hebrew, Spanish, Ar...",[\n 280\n (],[Michael Patrick King Productions],[6.7],"[77,915]",[255 user],[22 min],[2 Broke Girls ]
3,"[Kiefer Sutherland, Robert Cochran]",[USA],[Robert Cochran],"[\n , 6 November 2001 (USA)\n , \n ]",[195 episodes],[Action],"[English, Russian, German, Korean, Arabic, Spa...",[\n 231\n (],[20th Century Fox Television],[8.4],"[155,403]",[573 user],[44 min],[24 ]
4,[Tina Fey],[USA],[Tina Fey],"[\n , 11 October 2006 (USA)\n , \n ]",[138 episodes],[Comedy],[English],[\n 328\n (],[Broadway Video],[8.2],"[96,989]",[89 user],[22 min],[30 Rock ]
5,[John Lithgow],[USA],"[Bonnie Turner, Terry Turner]","[\n , 9 January 1996 (USA)\n , \n ]",[139 episodes],[Comedy],[English],[\n 749\n (],[Carsey-Werner Company],[7.8],"[40,250]",[89 user],"[25 min, 60 min]",[3rd Rock from the Sun ]
6,"[Rachael Taylor, David Wilcox]",[USA],[David Wilcox],"[\n , 2012 (USA)\n , \n ]",[13 episodes],[Drama],[English],"[\n 2,304\n (]",[Bonanza Productions],[6.9],"[16,200]",[44 user],[60 min],[666 Park Avenue ]
7,[Stephen Collins],[USA],[Brenda Hampton],"[\n , 26 August 1996 (USA)\n , \n ]",[243 episodes],[Drama],[English],[\n 386\n (],[Spelling Television],[5.1],"[21,794]",[270 user],[60 min],[7th Heaven ]
8,[Katey Sagal],[English],[Tracy Gamble],"[\n , 8 Simple Rules\n , \n ]",[76 episodes],[Comedy],[],[\n 555\n (],"[Flody Co., Touchstone Television]",[7.1],"[21,449]",[119 user],[30 min],[8 Simple Rules ]
9,[Shenae Grimes-Beech],[English],"[Darren Star, Jeff Judah, Gabe Sachs]","[\n , 90210: The Next Generation\n ...",[114 episodes],[Comedy],[],[\n 347\n (],"[CBS Television Studios, Sachs/Judah Productions]",[6.1],"[37,999]",[91 user],[],[90210 ]


In [442]:
#Saving as csv to avoid the need of runnig it all above again
df.to_csv('df.csv', header=True, index=False, encoding='utf-8') 

In [443]:
#Cleaning the data
#The scraped information originally was a list of lists. 
#Converting each cell from list item to item:
df = df.applymap(lambda x: ', '.join(x))

#Replacing the blank cells with np.nan
df = df.apply(lambda x: x.str.strip()).replace('', np.nan)

In [447]:
#Filling missing language with English because the search criteria to select the movies was 
#language to be English
df['language'].fillna('English', inplace=True)

In [449]:
#Removing unwanted characters from each column:
df['runtime'].replace(regex=True,inplace=True,to_replace=r'min',value=r'')
df['reviews'].replace(regex=True,inplace=True,to_replace=r'user',value=r'')
df['episodes'].replace(regex=True,inplace=True,to_replace=r'episodes',value=r'')
df['popularity'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
df['popularity'].replace(regex=True,inplace=True,to_replace=r'\(',value=r'')
df['date'].replace(regex=True,inplace=True,to_replace=r'\,',value=r'')
df['date'].replace(regex=True,inplace=True,to_replace=r'\n,',value=r'')
df['ratecount'].replace(regex=True,inplace=True,to_replace=r',',value=r'')

In [456]:
#Checking results:
df.head()

,cast,country,creator,date,episodes,genre,language,popularity,production,rate,ratecount,reviews,runtime,titles
0,"Aaron Stanford, Travis Fickett",USA,Travis Fickett,16 January 2015 (USA)\n,46,Adventure,English,310,Atlas Entertainment,7.6,"27,111",74,42,12 Monkeys
1,"Dylan Minnette, Brian Yorkey",USA,Brian Yorkey,31 March 2017 (USA)\n,21,Drama,English,43,July Moon Productions,8.5,"122,904",571,60,13 Reasons Why
2,"Kat Dennings, Whitney Cummings",USA,Whitney Cummings,19 September 2011 (USA)\n,137,Comedy,"English, Korean, Yiddish, Hebrew, Spanish, Ara...",280,Michael Patrick King Productions,6.7,"77,915",255,22,2 Broke Girls
3,"Kiefer Sutherland, Robert Cochran",USA,Robert Cochran,6 November 2001 (USA)\n,195,Action,"English, Russian, German, Korean, Arabic, Span...",231,20th Century Fox Television,8.4,"155,403",573,44,24
4,Tina Fey,USA,Tina Fey,11 October 2006 (USA)\n,138,Comedy,English,328,Broadway Video,8.2,"96,989",89,22,30 Rock


In [461]:
#Change dtypes to numeric on the numeric columns and filling nan or 0 with mean in some cases
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['runtime'] = df['runtime'].fillna(df['runtime'].mean())
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['rate'] = pd.to_numeric(df['rate'], errors='coerce')
df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce')
df['ratecount'] = pd.to_numeric(df['ratecount'])
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce').fillna(0).astype(int)

In [463]:
#Checking results
df

,cast,country,creator,date,episodes,genre,language,popularity,production,rate,ratecount,reviews,runtime,titles
0,"Aaron Stanford, Travis Fickett",USA,Travis Fickett,16 January 2015 (USA)\n,46,Adventure,English,310,Atlas Entertainment,7.6,27111,74,42.000000,12 Monkeys
1,"Dylan Minnette, Brian Yorkey",USA,Brian Yorkey,31 March 2017 (USA)\n,21,Drama,English,43,July Moon Productions,8.5,122904,571,60.000000,13 Reasons Why
2,"Kat Dennings, Whitney Cummings",USA,Whitney Cummings,19 September 2011 (USA)\n,137,Comedy,"English, Korean, Yiddish, Hebrew, Spanish, Ara...",280,Michael Patrick King Productions,6.7,77915,255,22.000000,2 Broke Girls
3,"Kiefer Sutherland, Robert Cochran",USA,Robert Cochran,6 November 2001 (USA)\n,195,Action,"English, Russian, German, Korean, Arabic, Span...",231,20th Century Fox Television,8.4,155403,573,44.000000,24
4,Tina Fey,USA,Tina Fey,11 October 2006 (USA)\n,138,Comedy,English,328,Broadway Video,8.2,96989,89,22.000000,30 Rock
5,John Lithgow,USA,"Bonnie Turner, Terry Turner",9 January 1996 (USA)\n,139,Comedy,English,749,Carsey-Werner Company,7.8,40250,89,42.087137,3rd Rock from the Sun
6,"Rachael Taylor, David Wilcox",USA,David Wilcox,2012 (USA)\n,13,Drama,English,0,Bonanza Productions,6.9,16200,44,60.000000,666 Park Avenue
7,Stephen Collins,USA,Brenda Hampton,26 August 1996 (USA)\n,243,Drama,English,386,Spelling Television,5.1,21794,270,60.000000,7th Heaven
8,Katey Sagal,English,Tracy Gamble,8 Simple Rules\n,76,Comedy,English,555,"Flody Co., Touchstone Television",7.1,21449,119,30.000000,8 Simple Rules
9,Shenae Grimes-Beech,English,"Darren Star, Jeff Judah, Gabe Sachs",90210: The Next Generation\n,114,Comedy,English,347,"CBS Television Studios, Sachs/Judah Productions",6.1,37999,91,42.087137,90210


In [464]:
#Saving file again to use in the modeling part
df.to_csv('df.csv', header=True, index=False, encoding='utf-8') 